<a href="https://colab.research.google.com/github/Saumya-Chauhan-MHC/ImageCode/blob/main/PyTorch_Resnet18_CIFAR10_Finetune_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the required libraries: Make sure Wandb is installed. Install it using pip:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.1/203.1 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.8 MB/s eta 0:00:00


In [2]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# Import the necessary libraries in the Python script
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms


In [4]:
#Initialize Wandb and login using your API key:
wandb.init(project='cifar-10_resnet18_pretrain_finetune', entity='saumya-s-chau')


wandb: Currently logged in as: saumya-s-chau. Use `wandb login --relogin` to force relogin


In [5]:
# Define your data transformations and create data loaders for CIFAR-10:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170498071/170498071 [00:12<00:00, 13177183.99it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
# Load the pre-trained ResNet18 model and modify the last fully connected layer to match the number of classes in CIFAR-10:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(classes))
model = model.to('cuda')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 241MB/s]


In [7]:
# Define the loss function and optimizer:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)


In [8]:
# Start the training loop and log the loss and accuracy using Wandb:

epochs = 10

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    train_loss /= len(trainloader)
    train_accuracy = 100.0 * correct / total

    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        test_loss /= len(testloader)
        test_accuracy = 100.0 * correct / total

        # Log the metrics to Wandb
        wandb.log({"epoch": epoch, "train_loss": train_loss, "train_accuracy": train_accuracy, 
                   "test_loss": test_loss, "test_accuracy": test_accuracy})

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%"
              f" - Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")

wandb.finish()


Epoch 1/10 - Train Loss: 1.1633 - Train Accuracy: 60.39% - Test Loss: 0.9061 - Test Accuracy: 68.77%
Epoch 2/10 - Train Loss: 0.8714 - Train Accuracy: 70.49% - Test Loss: 0.8269 - Test Accuracy: 72.84%
Epoch 3/10 - Train Loss: 0.7281 - Train Accuracy: 75.10% - Test Loss: 0.7425 - Test Accuracy: 74.83%
Epoch 4/10 - Train Loss: 0.6717 - Train Accuracy: 77.25% - Test Loss: 0.6745 - Test Accuracy: 77.01%
Epoch 5/10 - Train Loss: 0.6108 - Train Accuracy: 79.16% - Test Loss: 0.7316 - Test Accuracy: 75.97%
Epoch 6/10 - Train Loss: 0.5766 - Train Accuracy: 80.16% - Test Loss: 0.6297 - Test Accuracy: 79.16%
Epoch 7/10 - Train Loss: 0.5557 - Train Accuracy: 80.86% - Test Loss: 0.5685 - Test Accuracy: 80.75%
Epoch 8/10 - Train Loss: 0.5367 - Train Accuracy: 81.46% - Test Loss: 0.5853 - Test Accuracy: 80.52%
Epoch 9/10 - Train Loss: 0.5201 - Train Accuracy: 82.07% - Test Loss: 0.5464 - Test Accuracy: 81.34%
Epoch 10/10 - Train Loss: 0.5138 - Train Accuracy: 82.29% - Test Loss: 0.6475 - Test Accura

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁▃▄▆▅▇███▇
test_loss,█▆▅▃▅▃▁▂▁▃
train_accuracy,▁▄▆▆▇▇████
train_loss,█▅▃▃▂▂▁▁▁▁
epoch,9
test_accuracy,79.38
test_loss,0.64753
train_accuracy,82.29
train_loss,0.51382
